<h1><center>Введение в анализ данных</center></h1>
<hr>
<h2><center>Rossman Store Sales</center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

<h2>Подготовка данных:</h2>

<h3>Основная таблица</h3>

In [3]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/train.csv'
df = pd.read_csv(filepath, sep=',')
df.head()

/home/ivan/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


Заменим в StateHoliday буквы на цифры

In [4]:
df = df.replace({'StateHoliday': {'0': 0, 'a': 1, 'b': 2, 'c': 3}})

<h3>Дополнительная таблица</h3>

In [5]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/store.csv'
df_store = pd.read_csv(filepath, sep=',')
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


Заменяем в категориальных переменных значения на цифры

In [6]:
df_store = df_store.replace({'StoreType': {'a': 0, 'b': 1, 'c': 2, 'd': 3}})
df_store = df_store.replace({'Assortment': {'a': 0, 'b': 1, 'c': 2}})

Объединяем две таблицы

In [7]:
df_full = df.merge(df_store, on='Store', how='left')
df_full.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0,NaN,NaN,NaN


<h3>Тестовые данные</h3>

In [8]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/test.csv'
df_test = pd.read_csv(filepath, sep=',').set_index('Id')
df_test.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
Id,,,,,,,
1,1,4,2015-09-17,1.0,1,0,0
2,3,4,2015-09-17,1.0,1,0,0
3,7,4,2015-09-17,1.0,1,0,0
4,8,4,2015-09-17,1.0,1,0,0
5,9,4,2015-09-17,1.0,1,0,0


In [9]:
df_test = df_test.replace({'StateHoliday': {'0': 0, 'a': 1, 'b': 2, 'c': 3}})

В тестовых данных есть 11 объектов, у которых в столбце Open стоит nan. Заменим их на 1, и приведем весь столбец к целочисленному формату.

In [10]:
df_test.isnull().loc[:,'Open'].value_counts()

False    41077
True        11
Name: Open, dtype: int64

Заполняем пропуски в Open единицами (пропусков всего 11, так что это незначительно)

In [11]:
df_test.Open.fillna(value=1, inplace=True)
df_test['Open'] = df_test['Open'].astype(int)
df_test['Open'].unique()

array([1, 0])

Присоединяем дополнительную таблицу

In [12]:
df_test_full = df_test.merge(df_store, on='Store', how='left')
df_test_full.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,2015-09-17,1,1,0,0,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,3,4,2015-09-17,1,1,0,0,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,7,4,2015-09-17,1,1,0,0,0,2,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,8,4,2015-09-17,1,1,0,0,0,0,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,9,4,2015-09-17,1,1,0,0,0,2,2030.0,8.0,2000.0,0,NaN,NaN,NaN


In [13]:
del df
del df_store
del df_test

<h2>Формат выходных данных</h2>

In [13]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/sample_submission.csv'
df_out = pd.read_csv(filepath, sep=',')
print len(df_out)
df_out.head()

41088


,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


<h2>Random Forest</h2>

Попробуем лучше представить данные - вытащим дату, месяц, год

In [14]:
#train
df_full['Year'] = df_full['Date'].apply(lambda x: int(x[:4]))
df_full['Month'] = df_full['Date'].apply(lambda x: int(x[5:7]))
df_full['Day'] = df_full['Date'].apply(lambda x: int(x[8:10]))

#test
df_test_full['Year'] = df_test_full['Date'].apply(lambda x: int(x[:4]))
df_test_full['Month'] = df_test_full['Date'].apply(lambda x: int(x[5:7]))
df_test_full['Day'] = df_test_full['Date'].apply(lambda x: int(x[8:10]))

Введем переменную CompetitionOpen - открыт ли сейчас магазин конкурентов

In [15]:
df_full['CompetitionOpen'] = ((df_full['CompetitionOpenSinceYear'] < df_full['Year']) |\
                             ((df_full['CompetitionOpenSinceYear'] == df_full['Year']) &\
                              (df_full['CompetitionOpenSinceMonth'] <= df_full['Month']))).astype(int)

df_test_full['CompetitionOpen'] = ((df_test_full['CompetitionOpenSinceYear'] < df_test_full['Year']) |\
                             ((df_test_full['CompetitionOpenSinceYear'] == df_test_full['Year']) &\
                              (df_test_full['CompetitionOpenSinceMonth'] <= df_test_full['Month']))).astype(int)

In [16]:
df_full['FullDate'] = pd.to_datetime(df_full.Date, format='%Y-%m-%d')
df_test_full['FullDate'] = pd.to_datetime(df_test_full.Date, format='%Y-%m-%d')

import datetime
df_full['WeekNum'] = df_full['FullDate'].apply(func=lambda x: x.isocalendar()[1])
df_test_full['WeekNum'] = df_test_full['FullDate'].apply(func=lambda x: x.isocalendar()[1])

In [17]:
sum_sales = df_full.groupby('Store')['Sales'].sum()
sum_customers = df_full.groupby('Store')['Customers'].sum()
mean_check = sum_sales/sum_customers
df_meancheck = pd.DataFrame(mean_check, columns=['MeanCheck'])
df_meancheck = df_meancheck.reset_index()
df_full = df_full.merge(df_meancheck, on='Store', how='left')
df_test_full = df_test_full.merge(df_meancheck, on='Store', how='left')
del df_meancheck, sum_sales, sum_customers, mean_check

In [18]:
df_full['PromoInterval'].unique()

array([nan, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec'], dtype=object)

In [19]:
intervs_list = {'Jan,Apr,Jul,Oct': (1,4,7,10), 'Feb,May,Aug,Nov': (2,5,8,11), 'Mar,Jun,Sept,Dec': (3,6,9,12), np.nan: ()}

#train
df_full['Promo2Now'] = map(lambda week, month, year, promo2, interv, since_week, since_year:
                          int((promo2) == 1 & ((year > since_year) | ((year == since_year) & (week >= since_week))) &\
                           month in intervs_list[interv]), 
                           df_full['WeekNum'], df_full['Month'], df_full['Year'], df_full['Promo2'],
                           df_full['PromoInterval'], df_full['Promo2SinceWeek'], df_full['Promo2SinceYear'])

#test
df_test_full['Promo2Now'] = map(lambda week, month, year, promo2, interv, since_week, since_year:
                          int((promo2) == 1 & ((year > since_year) | ((year == since_year) & (week >= since_week))) &\
                           month in intervs_list[interv]), 
                           df_test_full['WeekNum'], df_test_full['Month'], df_test_full['Year'], 
                           df_test_full['Promo2'], df_test_full['PromoInterval'], df_test_full['Promo2SinceWeek'], 
                           df_test_full['Promo2SinceYear'])

In [20]:
df_full.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31,2015-07-31,2015-07-31,2015-07-31,2015-07-31
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,0,0,0,0,0
SchoolHoliday,1,1,1,1,1
StoreType,2,0,0,2,0


In [21]:
columns = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceYear', 'Year', 'Month', 'Day',
           'WeekNum', 'MeanCheck', 'Promo2Now']
X = np.array(df_full.loc[:,columns]).astype(int)
y = np.array(df_full.loc[:,'Sales']).astype(int)
X_test = np.array(df_test_full.loc[:,columns]).astype(int)

In [20]:
#попробуем предсказывать число покупателей, а прибыль получать из среднего чека
columns = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceYear', 'Year', 'Month', 'Day',
           'WeekNum', 'MeanCheck', 'Promo2Now']
X = np.array(df_full.loc[:,columns]).astype(int)
y = np.array(df_full.loc[:,'Customers']).astype(int)
X_test = np.array(df_test_full.loc[:,columns]).astype(int)

In [21]:
cls = RandomForestRegressor(n_estimators=50, criterion='mse').fit(X, y)

In [22]:
score = cls.predict(X_test)

In [24]:
score_final = score*df_test_full.MeanCheck

In [31]:
list(score_final)

[4143.2531905031065,
 7110.4965949923844,
 8971.8987436977677,
 7982.3594644466275,
 7318.4175501962682,
 5771.5302076832713,
 7430.9637319072863,
 8047.0897069300681,
 5286.8022040884998,
 5473.6102482349525,
 7156.6415468355553,
 8477.1757449782981,
 6642.198324352983,
 8958.843466867711,
 5447.0538330512718,
 4514.4843570446837,
 5655.8009188490169,
 9676.7567954824844,
 10636.019476791773,
 9815.1284669454417,
 7491.1728564331061,
 4429.7186406725095,
 5936.5413281127576,
 4163.0682310778748,
 7715.4775344942718,
 10098.896092828831,
 9294.4334578532489,
 5778.685379129759,
 5049.904782668089,
 4542.5336500849307,
 5381.178801039061,
 9909.9191004250024,
 6879.5050450859862,
 5509.4042593222121,
 5209.9827362522883,
 7622.317525041909,
 3498.6575585925311,
 7411.7041171330575,
 4379.4381023595279,
 7919.7537247594837,
 6817.6855441029966,
 5353.1642088228682,
 7956.1153402589925,
 6956.9031356528649,
 4455.0838422387842,
 6633.1921659047148,
 7346.2965804416399,
 11415.089470284132

In [25]:
features = {}
for i in xrange(len(columns)):
    features[columns[i]] = cls.feature_importances_[i]

In [26]:
import operator
sorted_features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('Open', 0.38031965986399285),
 ('MeanCheck', 0.1755108321297342),
 ('Store', 0.13082546894660058),
 ('CompetitionDistance', 0.12715432724823084),
 ('CompetitionOpenSinceYear', 0.058343360464486863),
 ('StoreType', 0.028985872715654792),
 ('Promo', 0.022680034001580644),
 ('DayOfWeek', 0.020823986259233967),
 ('WeekNum', 0.015278650918683735),
 ('Day', 0.01395824167387103),
 ('Assortment', 0.011337050537368797),
 ('Year', 0.0059066436669838861),
 ('Month', 0.0050762265898773393),
 ('SchoolHoliday', 0.0019021603841100402),
 ('Promo2Now', 0.00110714557860815),
 ('StateHoliday', 0.00079033902098206747)]

Лучшая точность:<br>
<br>
0.15051<br>
0.12415<br>

<h2>Boosting</h2>

In [36]:
%%timeit
boost = GradientBoostingClassifier(n_estimators=50).fit(X[:100], y[:100])

1 loop, best of 3: 3.53 s per loop


Очень долго работает - с учетом асимптотики на всей выборке будет обучаться больше года! 

<h2>Линейная регрессия</h2>

Векторизуем остальные категориальные признаки: DayOfWeek, StoreType, Assortment, и нормализуем вещественный CompetitionDistance

In [23]:
df_full_linear = df_full.copy()
day_of_week = OneHotEncoder().fit_transform(df_full_linear['DayOfWeek'].values.reshape(-1, 1)).toarray().astype(int)
store_type = OneHotEncoder().fit_transform(df_full_linear['StoreType'].values.reshape(-1, 1)).toarray().astype(int)
assortment = OneHotEncoder().fit_transform(df_full_linear['Assortment'].values.reshape(-1, 1)).toarray().astype(int)

for i in xrange(len(day_of_week[0])):
    df_full_linear['day_of_week_{}'.format(i)] = day_of_week[:,i]
    
for i in xrange(len(store_type[0])):
    df_full_linear['store_type_{}'.format(i)] = store_type[:,i]
    
for i in xrange(len(assortment[0])):
    df_full_linear['assortment_{}'.format(i)] = assortment[:,i]
    
df_full_linear.drop(df_full_linear[['DayOfWeek', 'StoreType', 'Assortment']], axis=1, inplace=True)

In [24]:
df_full_linear.head().T

,0,1,2,3,4
Store,1,2,3,4,5
Date,2015-07-31,2015-07-31,2015-07-31,2015-07-31,2015-07-31
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,0,0,0,0,0
SchoolHoliday,1,1,1,1,1
CompetitionDistance,1270,570,14130,620,29910
CompetitionOpenSinceMonth,9,11,12,9,4


In [25]:
df_full_linear.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 37 columns):
Store                        1017209 non-null int64
Date                         1017209 non-null object
Sales                        1017209 non-null int64
Customers                    1017209 non-null int64
Open                         1017209 non-null int64
Promo                        1017209 non-null int64
StateHoliday                 1017209 non-null int64
SchoolHoliday                1017209 non-null int64
CompetitionDistance          1014567 non-null float64
CompetitionOpenSinceMonth    693861 non-null float64
CompetitionOpenSinceYear     693861 non-null float64
Promo2                       1017209 non-null int64
Promo2SinceWeek              509178 non-null float64
Promo2SinceYear              509178 non-null float64
PromoInterval                509178 non-null object
Year                         1017209 non-null int64
Month                        1017209 non-null in

In [26]:
columns = ['Store', 'Promo', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance', 'Promo2',
          'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
          'day_of_week_6', 'store_type_0', 'store_type_1', 'store_type_2', 'store_type_3',
          'assortment_0', 'assortment_1', 'assortment_2', 'MeanCheck']
X_linear = np.array(df_full_linear.loc[df_full_linear.Open == 1,columns])
y_linear = np.array(df_full_linear.loc[df_full_linear.Open == 1,'Sales'])

In [27]:
X_linear.shape

(844392, 21)

Избавимся от Nan в CompetitionDistance - заполним нулями

In [32]:
np.unique(np.isnan(X_linear[:,4]))

array([False,  True], dtype=bool)

In [33]:
X_linear[np.isnan(X_linear[:,4]),4] = 0

In [34]:
np.unique(np.isnan(X_linear[:,4]))

array([False], dtype=bool)

Отнормируем данные

In [35]:
X_linear = StandardScaler().fit_transform(X_linear)
#y_linear = (y_linear - y_linear.mean())/y_linear.std()

In [36]:
y_linear[0]

5263

Аналогично преобразуем тестовые данные

In [42]:
df_linear_test = df_test_full.copy()
day_of_week = OneHotEncoder().fit_transform(df_linear_test['DayOfWeek'].values.reshape(-1, 1)).toarray().astype(int)
store_type = OneHotEncoder().fit_transform(df_linear_test['StoreType'].values.reshape(-1, 1)).toarray().astype(int)
assortment = OneHotEncoder().fit_transform(df_linear_test['Assortment'].values.reshape(-1, 1)).toarray().astype(int)

for i in xrange(len(day_of_week[0])):
    df_linear_test['day_of_week_{}'.format(i)] = day_of_week[:,i]
    
for i in xrange(len(store_type[0])):
    df_linear_test['store_type_{}'.format(i)] = store_type[:,i]
    
for i in xrange(len(assortment[0])):
    df_linear_test['assortment_{}'.format(i)] = assortment[:,i]
    
df_linear_test.drop(df_linear_test[['DayOfWeek', 'StoreType', 'Assortment']], axis=1, inplace=True)

df_linear_test.loc[:, 'CompetitionDistance'] = (df_linear_test.loc[:, 'CompetitionDistance'] \
                                                - df_linear_test.loc[:, 'CompetitionDistance'].mean()).div(\
                                                 df_linear_test.loc[:, 'CompetitionDistance'].std())

columns = ['Store', 'Promo', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance', 'Promo2',
          'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
          'day_of_week_6', 'store_type_0', 'store_type_1', 'store_type_2', 'store_type_3',
          'assortment_0', 'assortment_1', 'assortment_2', 'MeanCheck']
X_linear_test = np.array(df_linear_test.loc[:,columns])

X_linear_test[np.isnan(X_linear_test[:,4]),4] = 0
X_linear_test = StandardScaler().fit_transform(X_linear_test)

In [72]:
lin_reg = LinearRegression().fit(X_linear, y_linear)

In [38]:
ridge = Ridge(alpha=1.0).fit(X_linear, y_linear)

Очень плохой R^2, линейная регрессия не подходит

In [73]:
lin_reg.score(X_linear, y_linear)

0.2199726187659895

In [39]:
ridge.score(X_linear, y_linear)

0.22576761266121803

In [43]:
score = ridge.predict(X_linear_test)

In [44]:
ridge.coef_

array([   31.81852465,  1141.6415485 ,     9.83301624,    86.53788112,
        -235.65130941,  -450.63230753,   367.87208946,   -27.99085019,
        -162.53160218,  -157.45845727,   -19.64746939,    -2.49174034,
          20.23351473,    27.95934458,   734.51918165,    -5.5536175 ,
        -240.41478648,  -169.65306987,  -293.5647647 ,   227.63767254,
         334.5625893 ])

In [45]:
score[X_linear_test[:,1] == 0] = 0

Лучшая точность:<br>
<br>
0.43265<br>
0.43062<br>

<h2>SVM</h2>

In [62]:
%%timeit
svr = SVR().fit(X_linear[:10000], y_linear[:10000])

1 loop, best of 3: 9.62 s per loop


In [65]:
svr = SVR().fit(X_linear[750000:], y_linear[750000:])

In [71]:
svr.score(X_linear[750000:], y_linear[750000:])

0.093636867303272933

In [66]:
score = svr.predict(X_linear_test)

In [67]:
score[df_test.Open == 0] = 0

/home/ivan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


<h2>Запись в файл</h2>

In [32]:
df_out = pd.DataFrame(score, columns=['Sales'])
df_out = df_out.reset_index().rename(index=str, columns={'index': 'Id'})
df_out.loc[:,'Id'] += 1
df_out.head()

,Id,Sales
0,1,4143.253191
1,2,7110.496595
2,3,8971.898744
3,4,7982.359464
4,5,7318.417550


In [33]:
df_out['Sales'].isnull().unique()

array([False], dtype=object)

In [34]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/rossman.csv'
df_out.to_csv(filepath, index=False)

Проверка, что формат правильный:

In [52]:
filepath = '/media/ivan/Acer/Ivan/sphere/Rossman/rossman.csv'
df_out = pd.read_csv(filepath)
df_out.head()

,Id,Sales
0,1,4731.966667
1,2,7655.700000
2,3,8692.633333
3,4,7415.166667
4,5,7899.866667


In [53]:
len(df_out)

41088